# Explainers Notebook

In [ ]:
#imports used for python code
import networkx as nx
from wordcloud import WordCloud
from PIL import Image
from colour import Color
import pandas as pd
import re
import numpy as np
import io
import nltk
import json

%matplotlib inline
from fa2 import ForceAtlas2
import matplotlib
import matplotlib.pyplot as plt
from community import community_louvain

 ## Motivation 

### What is your dataset?

The data we decided to use for our analysis, contains the Star Wars Movie Transcipts for all of the episodes (1-8). And in addition to this we have also downloaded all of the Wookiepedia pages for the charcters who are part of the canonical story. This means the characters from the 8 main movies (Episode 1-8), the two stand alone movies (Rogue One and Solo a Star Wars Story) and the two animated series (The Clones Wars and Rebels). This ends up being a total of 408 Wookiepedia articles. The total size of the data is 12.8 MB.
    

### Why did you choose this/these particular dataset(s)?

We decided on using these two datasets because they hold all the information we need to get out the information we want. The transcript we choose as they hold all the information given in the movies, which is what most casual Star Wars fans only has knowledge of. These are good for analysing sentiments and other stuff.
However, it is very difficult to get any sort of relation between the characters out of the scripts therefore we decided to also download the Wookiepedia articles, as these through the use of hyperlinks will tell us how the characters are connected, according to the fans writing these articles. Also from the Wookiepedia articles we also get some extra data about characters such as race and affiliations, which might not be available through the transcripts.
    
### What was your goal for the end user's experience?

The goal of the project to get a insight to the universe which might not be obvious to the average fan, through analyzing communities, sentiments and nodedegrees. Hopefully we will be able to get some knowledge which will enlighten and educate the user in how the universe is actually working.

## The Dataset
 
### Data cleaning and preprocessing

As our data consist of two sets, we will first explain how we got and preprocessed the movie transcripts, and afterwards for the Wookiepedia articles.

The moivetranscripts come from two sources, IMSDb (https://www.imsdb.com) and the Transcripts wiki (http://transcripts.wikia.com/wiki/Transcripts_Wiki). This is due to neither of the two having all of the scripts, at the time of writing this. The scripts from IMSDB we first downloaded as HTML files, and then removed all of the HTML code to only get the script text. This was done using regular expressions. The scripts we found on the transcripts wiki, has been downloaded as other wiki data through the use of an API. When we had all of the text dataloaded, we had to do some more stuff that Lasse knows more about. (Should we have the preprocessing code here???)

For the Wookiepedia pages we first had to get a list of all the canonical characters. Unfortunately Wookiepedia does not have such a list, and we had to find it on the Wikipedia page for List of Star Wars characters (https://en.wikipedia.org/wiki/List_of_Star_Wars_characters). This however posed a few unexpected challenges. First of all we downloaded the page through the use of the wikipedia API. Most of the characters names was the written in a set of brackets like {{visible anchor | name}}. This made it fairly easy to the the names by using some regular expressions. However not all of the names in the list are the same as the names used on Wookiepedia. This meant we had to removed titles such as Admiral, or only use the first or last name of a character. This however resulted in us getting to many names, as some of the last names of characters, like (Jabba the) Hutt, is the name of a species and therefore has its own Wookiepedia page. These we ended up removing manually. The last hurdle when working with the Wikipedia page, was that two characters names did not follow the earlier mentioned structure, meaning we had to add these manually as well. These two where Anakin Skywalker and Darth Sidious, and as you will learn later are rather important for the network. In the end we ended up with 407 names, which have all been saved in a CSV file for convinence. 

In addition to the names we also add the race of the characters and their "goodness" in the CSV file. The race was found on the Wokkiepedia page, and the good is calculated on a scale based on how many an which affiliations the characters have on their pages. Where an evil affiliation will count as -1 and a good affiliation counts as +1. Affiliations being good or evil is based on whether they are connected to the Sith (evil) or the Jedi(good). (Again should we add the preprocessing code here???)
    


### Dataset stats

So in total we ended up with a CSV file containing 408 rows and four columns (Name, Wookiepedia link, Race and Goodness), 407 Wookiepedia articles and 8 movies transcripts. This total up to 12.8 MB of data. 

 ## Tools, theory and analysis
 
     Talk about how you've worked with text, including regular expressions, unicode, etc.
 
     Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.
     
    How did you use the tools to understand your dataset?

 
### Overall idea
 
We want to create a network over the characters and their connections. This graph will contain the character names as nodes and the edges between them will be the hyperlinks from the Wookiepedia articles. Where an edge from one node to another means that that charcters links to the other. From the network we wish to anylise the centrality measures to ???. We also want to find communities in the network and see how these fit up with the known alliances, races or movies. In addition to this we also wish to use the Wookiepedia articles to make a sentiment analysis to see if there is a race which we can consider to be more happy than the others, and which is the most negative.

The movie transcript will be used for sentiment analysis, to try to see if we can detect certain scenes in the movies and see how the mood changes over a single movie or all of the movies. We also wish to see how the mood for individual characters change over a single movie or multiple movies.  
 

 

### Analysis step 1 : The Network
 
#### Explain what you're interested in
     
Interested in creating a network over the characters, to analyze connections to see who is the most important person, analyze communities to see if these follow any grouping of the actual connection such as race, movie, alliance, family or other. And we wish to find which race is the most happy, based on the content of the Wookiepedia artclies for the characters of each race.

To create the network we will create a directed graph, where we will first create a nodes for each of the characters from our CSV file. As a attribute each node will get their race and goodness. Then we will load each file one by one and add an edge from a node to another node if the corresponding Wookiepedia file from the first node contains a hyperlink to the other node. The way we get the links is by using regular expressions as all of the links are trapped between two set of square brackets, with the link match the name of the character. 

We will then draw the graph using the ForceAtlas library, with the nodes sizes according to their degree, and colored accoring to their goodneww score from evil red to good green.


In [ ]:
# Loading in the character data
df = pd.read_csv('starwarscharacters.csv')

# Creating the directed graph for the network
G = nx.DiGraph()

# Adding the nodes
for i in range(len(df)):
    name = df.loc[i]['name']
    species = df.loc[i]['species']
    goodness = df.loc[i]['goodness']
    G.add_node(name, goodness=goodness, species=species)
    
# Adding the edges
for i in range(len(df)):
    name = df.loc[i]['name']
    
    # Loading the Wookieepedia text
    fileName = name.replace(" ", "_") + ".txt"
    text = open("./Wookiepediafiles/" + fileName, "r").read()
    
    # Finding the links in the Wookieepedia article and adding them as edges
    links = re.findall('\[\[(.*?)\]\]', text)
    for link in links:
        for l in link.split("|"):
            if l in list(df['name']):
                if not G.has_edge(name, l):
                    G.add_edge(name, l)
                break

                

In [ ]:
# Set up forceatlas2 parameters
G_undir = G.to_undirected()

forceatlas2 = ForceAtlas2(# Behavior alternatives
                          outboundAttractionDistribution=False,  # Dissuade hubs
                          linLogMode=False,
                          adjustSizes=False,
                          edgeWeightInfluence=1.0,

                          # Performance
                          jitterTolerance=1.0,  # Tolerance
                          barnesHutOptimize=False,
                          barnesHutTheta=1.2,
                          multiThreaded=False,  # NOT IMPLEMENTED

                          # Tuning
                          scalingRatio=1.0,
                          strongGravityMode=False,
                          gravity=100.0,
                          # Log
                          verbose=True)
positions = forceatlas2.forceatlas2_networkx_layout(G_undir, pos=None, iterations=10000)

In [ ]:
# Drawing the graph
plt.figure(figsize=(20,12))
plt.title("Node size scaled by degree")


# Creating the colorscale according to goodness
red = 255
green = 0
stepSize = 64 
colorScale = []
while green < 255 :
    green += stepSize
    if green > 255 : 
        green = 255
    colorScale.append(('#%02x%02x%02x' % (red, green, 0)))

while(red > 0) :
    red -= stepSize
    if red < 0 :
        red = 0
    colorScale.append(('#%02x%02x%02x' % (red, green, 0)))

# Setting up datastructure used for the drawing    
sizemap_degree = []    
colormap = []
for node in G_undir:
    sizemap_degree.append(G.degree(node, weight="weight") + 2)
    idx = G.node[node]['goodness']+3
    colormap.append(colorScale[idx])

nodelist = [node for node in G_undir.nodes]
edgelist = [edge for edge in G_undir.edges]

# Adding labels for the most important characters
labelNodes = ["Anakin Skywalker", "Obi-Wan Kenobi", "R2-D2", "C-3PO", "Darth Sidious", "Luke Skywalker", "Han Solo"]
labels = {}
for node in nodelist :
    if node in labelNodes :
        labels[node] = node
    else :
        labels[node] = ""

# Drawing the nodes, edges and selected labels
nx.draw_networkx_nodes(G_undir, positions, with_labels=False, nodelist=nodelist, node_color=colormap, edgecolors="black", node_size=sizemap_degree)
nx.draw_networkx_edges(G_undir, positions, alpha=0.2, edge_color="black", width=0.5, edgelist=edgelist)
nx.draw_networkx_labels(G_undir,pos=positions, labels = labels, font_color = "black", font_weight = "bold")
plt.show()

### Analysis step 1.1 : Centrality measurements
 
#### Explain what you're interested in

We are interested in analyzing the network with different centrality measurements. The goal of this is to attempt to grade all our characters based on their importance in the star wars universe.  We will use the betweenness centrality measurement, eigenvector centrality measurement and the degree centrality measurement for this purpose. 

 
#### Explain the tool
 
#### Apply the tool

blablabla
 
#### Discuss the outcome

### Analysis step 1.2 : Community detection
 
#### Explain what you're interested in
For this step we wish to analyze our network to see if we can find any communities. And if so how to fit with the actual affiliations between the characters such as family, race, movie or alliance. Or see if we find some communities which are not obvious from the movies.
 
#### Explain the tool
For this we will be using the Louvain algorithm to find the communities. The way this algorithm works is by trying to maximize the modularity, which is a measurement of how many links are between the nodes in the community compare to between the different communities. 
 
#### Apply the tool
partition = community_louvain.best_partition(G_undir)
 
#### Discuss the outcome
We get that the modularity is ???

### Analysis step 1.3 : Race sentiment analysis
 
#### Explain what you're interested in
 
#### Explain the tool
 
#### Apply the tool
 
#### Discuss the outcome

 ### Analysis step 2 : The Transcripts
 
 #### Explain what you're interested in
 
 #### Explain the tool
 
 #### Apply the tool
 
 #### Discuss the outcome

 ### Analysis step 2.1 : Sentiments
 
 #### Explain what you're interested in
 
 #### Explain the tool
 
 #### Apply the tool
 
 #### Discuss the outcome

 ## Discussion 
 